In [1]:
from datetime import datetime
from planet import Auth
from planet import Session, data_filter
import json
import time
import os

# We will also create a small helper function to print out JSON with proper indentation.
def indent(data):
    print(json.dumps(data, indent=2))

In [2]:
# if your Planet API Key is not set as an environment variable, you can paste it below
if 'PL_API_KEY' in os.environ:
    API_KEY = os.environ['PL_API_KEY']
else:
    API_KEY = 'PLAK03522c42c1ee4651a5b0e983b599f1c7'
    os.environ['PL_API_KEY'] = API_KEY

client = Auth.from_key(API_KEY)

In [3]:
with open("map.geojson") as f:
    geom = json.loads(f.read())

In [5]:
# Define the filters we'll use to find our data

item_types = ["PSScene"]

geom_filter = data_filter.geometry_filter(geom)
clear_percent_filter = data_filter.range_filter('clear_percent', 90)
date_range_filter = data_filter.date_range_filter("acquired", gt = datetime(month=1, day=1, year=2017))
cloud_cover_filter = data_filter.range_filter('cloud_cover', None, 0.1)

combined_filter = data_filter.and_filter([geom_filter, clear_percent_filter, date_range_filter, cloud_cover_filter])

In [8]:
async with Session() as sess:
    cl = sess.client('data')
    request = await cl.create_search(name='Vodamulaa_Maldives_test',search_filter=combined_filter, item_types=item_types)

In [10]:
# Search the Data API

# The limit paramter allows us to limit the number of results from our search that are returned.
# The default limit is 100. Here, we're setting our result limit to 50.
async with Session() as sess:
    cl = sess.client('data')
    items = cl.run_search(search_id=request['id'], limit=50)
    item_list = [i async for i in items]

In [ ]:
for item in item_list:
    print(item['id'], item['properties']['item_type'])

In [12]:
path_ouput = os.path.join(os.getcwd(), 'data', 'coastsat_data', 'Vodamulaa_Maldives', 'PSScene')
with open(os.path.join(path_ouput, 'results.json'),'w') as f:
    jsonStr = json.dumps(item_list)
    f.write(jsonStr)
    f.close()

In [15]:
# Define our item_id, and item_type
item = item_list[0]
item_id = item['id']
item_type = item['properties']['item_type']

In [18]:
# Analytic Asset
async with Session() as sess:
    cl = sess.client('data')
    # Get Asset
    asset_desc = await cl.get_asset(item_type_id=item_type,item_id=item_id, asset_type_id='ortho_analytic_4b')
    # Activate Asset
    await cl.activate_asset(asset=asset_desc)
    # Wait Asset
    await cl.wait_asset(asset=asset_desc)
    # Download Asset
    asset_path = await cl.download_asset(asset=asset_desc, directory=path_ouput, overwrite=True)

c:\Users\mp222\OneDrive - Imperial College London\IslandTimeGitHub\IslandTime\data\coastsat_data\Vodamulaa_Maldives\PSScene\20231023_052057_13_2483_3B_AnalyticMS.tif: 100%|██████████| 467k/467k [00:12<00:00, 38.9MB/s] 


In [ ]:
# Analytic XML Asset
async with Session() as sess:
    cl = sess.client('data')
    # Get Asset
    asset_desc = await cl.get_asset(item_type_id=item_type,item_id=item_id, asset_type_id='ortho_analytic_4b_xml')
    # Activate Asset
    await cl.activate_asset(asset=asset_desc)
    # Wait Asset (this may take some time!)
    await cl.wait_asset(asset=asset_desc)
    # Download Asset
    asset_path = await cl.download_asset(asset=asset_desc, directory=path_ouput, overwrite=True)